# Table Creation

We have previously covered a basic CREATE TABLE statement, creating an empty shell of a table in the tempDB database.

We did this as follows:

In [ ]:
create table #myFirstTempTable
(
    ID int,
    miscValue varchar(50)
);

select *
from #myFirstTempTable;

However, there is much more that we can achieve with the CREATE TABLE statement in terms of column behaiour and actions, such as:

-   Identity Columns
-   Default Values
-   Computed Columns

#### Identity Columns

If you remember back to our work with Window Functions in which we used ROW\_NUMBER to create an auto incrementing value for our data, this is a very similar principle except that our table is creating this incrementing value for us as part of it's definition.

We add `IDENTITY(startValue, increment)` to the column in question as part of the table definition.

In order to see how this would look in a create statement we shall edit the temporary statement above to ensure that the ID column is auto incrementing, starting at number 1 and increasing by 1 each time:

In [ ]:
drop table if exists #myFirstTempTable;

create table #myFirstTempTable
(
    ID int identity(1, 1),
    miscValue varchar(50)
);

As implied by the definition and unlike with ROW_NUMBER, we are allowed to amend the values used by the IDENTITY column.

For example, imagine you are creating a database for a brand new gym and wish to create a Members table.  In the real world we would be unlikely to see a member ever given MemberNo 1, it is more likely we would start at 1 million and increment in 5s so that the gym quickly appears to be a popular and healthy business.

Let's make this adjustment now:

In [ ]:
drop table if exists #myFirstTempTable;

create table #myFirstTempTable
(
    MemberNumber int identity(1000000, 5),
    miscValue varchar(50)
);

#### Default Values

Sticking with the gym motif we can maybe make some more changes to our temporary table.

We shall leave MemberNumber in place but include firstName, lastName, and add a BIT column (true or false represented as 1 or 0 respectively) to indicate whether a member number is active or not:

In [ ]:
drop table if exists #memberTable;

create table #memberTable
(
    memberNumber int identity(1000000, 5),
    firstName varchar(25),
    lastName varchar(35),
    fullName varchar(60),
    isActive bit
);

It makes sense though, at this point, to consider the application which the staff would use in order to create new members in the gym system (we rarely build databases in isolation and sometimes need to consider external applications and and requirements they may have).

It would seem unnecessary to require a staff member to input "active" each time they create a new member and therefore we can tell SQL Server to default to 1 if a value is not presented to it.

This is called a Default Value.

We can create this in a couple of different ways, firstly as part of the table definition:

In [ ]:
drop table if exists #memberTable;

create table #memberTable
(
    memberNumber int identity(1000000, 5),
    firstName varchar(25),
    lastName varchar(35),
    fullName varchar(60),
    isActive bit default(1)
);

A Default can also be added post creation:

In [ ]:
drop table if exists #memberTable;

create table #memberTable
(
    memberNumber int identity(1000000, 5),
    firstName varchar(25),
    lastName varchar(35),
    fullName varchar(60),
    isActive bit
);

alter table #memberTable
    add constraint df_isActiveMember
    default(1)
    for isActive;

From the above code we can see the format is as follows:

<code>alter table <i>tableName</i><br/>
&nbsp&nbsp&nbsp&nbspadd constraint <i>constraintName</i><br/>
&nbsp&nbsp&nbsp&nbspdefault(<i>defaultValue</i>)<br/>
&nbsp&nbsp&nbsp&nbspfor <i>columnName</i></code>

This latter method, although more complex in terms of syntax, allows us to name the constraint whereas the former will let SQL Server provide a name.

This can be a useful thing for DBAs to see when then are performing maintenance and providing data dictionaries and is therefore the preferred method.

#### Calculated Columns

We are going to keep persevering with our basic gym membership table and now consider another potential improvement to our database.  Consider the following:

After a few weeks of testing we have found that our sales personnel are not very good at typing names into boxes.  They can accurately type first and last names, but we have seen frequent spelling mistakes and inconsistencies between those and the fullName field.

We wish to eliminate this issue within the database itself, taking the onus away from the employee.

This can be achieved through the use of Calculated Columns, allowing the database to define the column and no longer requiring any input from the user:

In [ ]:
drop table if exists #memberTable;

create table #memberTable
(
    memberNumber int identity(1000000, 5),
    firstName varchar(25),
    lastName varchar(35),
    fullName as concat_ws(' ', firstName, lastName),
    isActive bit
);

alter table #memberTable
    add constraint df_isActiveMember
    default(1)
    for isActive;

As you can see, we do not need to provide a datatype for a calculated column as SQL Server determines it from the constituent columns.

The format of a Calculated Column (also known as Computed Column) is as follows:

<code><i>columnName</i> as <i>columnDefinition</i></code>